<a href="https://colab.research.google.com/github/Elwing-Chou/ml1216/blob/main/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
path = "/content/drive/MyDrive/additional/train.zip"
f = zipfile.ZipFile(path)
f.extractall()

In [ ]:
import glob
import pandas as pd
dogs = glob.glob("train/dog.*.jpg")
cats = glob.glob("train/cat.*.jpg")
df = pd.DataFrame({
    "path":dogs+cats,
    "target":[0]*len(dogs)+[1]*len(cats) 
})
df

In [ ]:
import random
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
idx = random.randint(0, len(df)-1)
img = load_img(df["path"][idx], target_size=(224, 224))
plt.imshow(img)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dropout, Dense, GlobalAveragePooling2D
# compile前就要做: layer.trainable = False
for l in vgg.layers:
    l.trainable = False
layers = [
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.25),
    Dense(2, activation="softmax")
]
layers = vgg.layers + layers
model = Sequential(layers)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
x, y = np.array(df["path"]), np.array(df["target"])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [ ]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
# https://github.com/keras-team/keras-applications/tree/master/keras_applications
from tensorflow.keras.applications.vgg16 import preprocess_input

def get_data(x, y, batch=20):
    idx = np.random.randint(0, len(x), size=batch)
    imgs, imgs_pre, ans = [], [], y[idx]
    for p in x[idx]:
        img = load_img(p, target_size=(224, 224))
        img = np.array(img)
        imgs.append(img)
        imgs_pre.append(preprocess_input(img))
    imgs, imgs_pre, ans = np.array(imgs), np.array(imgs_pre), np.array(ans)
    return (imgs, imgs_pre, ans)
a, b, c = get_data(x, y)
print(a.shape, b.shape, c.shape)